In [1]:
import requests
import json
import time
import pandas as pd
import mysql.connector
from IPython.display import clear_output
from datetime import datetime
from config import api_football_key, conn_host, conn_database, conn_user, conn_password
import os

In [2]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

In [3]:
def get_winner(home_score, away_score):
    if home_score > away_score:
        return 'H'
    elif away_score > home_score:
        return 'A'
    else:
        return 'D'

In [4]:
start_season = 2014
end_season = 2022

In [5]:
fixtures_df = execute_query(f"SELECT m.id, m.date, m.season, l.name AS league, ht.id as home_id, at.id as away_id, ht.name as home_team, at.name as away_team, m.home_score, m.away_score, m.home_odds, m.away_odds, m.draw_odds FROM matches AS m INNER JOIN teams AS ht ON (m.home_id = ht.id) INNER JOIN teams AS at ON (m.away_id = at.id) INNER JOIN leagues AS l ON (m.league_id = l.id) WHERE (m.home_odds IS NOT NULL AND m.season >= {start_season} AND m.season <= {end_season}) ORDER BY m.date ASC")
fixtures_df['winner'] = fixtures_df.apply(lambda x: get_winner(x['home_score'], x['away_score']), axis=1)

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
fixtures_df.tail()

,id,date,season,league,home_id,away_id,home_team,away_team,home_score,away_score,home_odds,away_odds,draw_odds,winner
3201,838156,2022-07-17 11:00:00,2022,Serie A (Brazil),152,151,Juventude,Goias,0,0,2.04,4.10,3.14,D
3202,838153,2022-07-17 16:00:00,2022,Serie A (Brazil),126,124,Sao Paulo,Fluminense,2,2,2.53,3.11,3.03,D
3203,838155,2022-07-17 18:00:00,2022,Serie A (Brazil),120,1062,Botafogo RJ,Atletico-MG,0,1,4.94,1.78,3.49,A
3204,838158,2022-07-17 18:00:00,2022,Serie A (Brazil),144,154,Atletico GO,Fortaleza,0,1,2.07,3.80,3.27,A
3205,838151,2022-07-17 19:00:00,2022,Serie A (Brazil),125,794,America MG,Bragantino,0,3,2.77,2.70,3.11,A


In [7]:
n_last_games = 5

In [8]:
def get_games_results(games, cenario):
    loser = 'A' if cenario == 'H' else 'H'
    return len(games.loc[games['winner'] == cenario].index), len(games.loc[games['winner'] == 'D'].index), len(games.loc[games['winner'] == loser].index)

def get_goals_mean(games, team_id, cenario):
    games = games.iloc[-n_last_games:,:]
    
    home_games = games.loc[(games['home_id'] == team_id)]
    away_games = games.loc[(games['away_id'] == team_id)]
    total_games = len(home_games.index) + len(away_games.index)
    
    home_scored_goals = home_games['home_score'].sum()
    away_scored_goals = away_games['away_score'].sum()
    total_scored_goals = home_scored_goals + away_scored_goals
    
    home_conceded_goals = home_games['away_score'].sum()
    away_condeded_goals = away_games['home_score'].sum()
    total_conceded_goals = home_conceded_goals + away_condeded_goals
    
    return_list = [total_scored_goals / total_games, total_conceded_goals / total_games]
    if cenario == 'H':
        return_list.extend([home_scored_goals / len(home_games.index), home_conceded_goals / len(home_games.index)])
    else:
        return_list.extend([away_scored_goals / len(away_games.index), away_condeded_goals / len(away_games.index)])
    
    return return_list
    

def get_team_previous_games_stats(team_id, season, game_date, cenario):
    previous_games = fixtures_df.loc[((fixtures_df['home_id'] == team_id) | (fixtures_df['away_id'] == team_id)) & (fixtures_df['date'] < game_date) & (fixtures_df['season'] == season)]
    home_games = previous_games.loc[(previous_games['home_id'] == team_id)]
    away_games = previous_games.loc[(previous_games['away_id'] == team_id)]
    
    total_games = len(home_games.index) + len(away_games.index)
    if total_games < 10:
        return
    
    home_wins, home_draws, home_losses = get_games_results(home_games, 'H')
    away_wins, away_draws, away_losses = get_games_results(away_games, 'A')
    
    total_wins = home_wins + away_wins
    total_draws = home_draws + away_draws
    total_losses = home_losses + away_losses
    
    win_pct = total_wins * 100 / total_games
    draw_pct = total_draws * 100 / total_games
    loss_pct = total_losses * 100 / total_games
    
    points_achieved = total_wins * 3 + total_draws
    points_pct = (points_achieved * 100) / (total_games * 3)
    
    if cenario == 'H':
        ha_win_pct = home_wins * 100 / len(home_games.index)
        ha_draw_pct = home_draws * 100 / len(home_games.index)
        ha_loss_pct = home_losses * 100 / len(home_games.index)
    else:
        ha_win_pct = away_wins * 100 / len(away_games.index)
        ha_draw_pct = away_draws * 100 / len(away_games.index)
        ha_loss_pct = away_losses * 100 / len(away_games.index)
        
    scored_mean, conceded_mean, ha_scored_mean, ha_conceded_mean = get_goals_mean(previous_games, team_id, cenario)
    
    return [points_pct, win_pct, draw_pct, loss_pct, ha_win_pct, ha_draw_pct, ha_loss_pct, scored_mean, conceded_mean, ha_scored_mean, ha_conceded_mean]
        

In [9]:
data_model = []
for index, game in fixtures_df.iterrows():
    clear_output(wait=True)
    print("{}/{}".format(index, len(fixtures_df.index)))
    
    home_stats = get_team_previous_games_stats(game['home_id'], game['season'], game['date'], 'H')
    if not home_stats:
        continue
        
    away_stats = get_team_previous_games_stats(game['away_id'], game['season'], game['date'], 'A')
    if not away_stats:
        continue
        
    data_model.append([game['id'], game['date'], game['season'], game['home_team'], game['away_team'], game['home_odds'], game['away_odds'], game['draw_odds']] + home_stats + away_stats + [game['winner']])

3205/3206


In [10]:
print(data_model[100])

[190648, Timestamp('2014-09-13 18:30:00'), 2014, 'Chapecoense-SC', 'Sport Recife', 2.03, 3.88, 3.24, 33.333333333333336, 25.0, 25.0, 50.0, 40.0, 30.0, 30.0, 0.6, 1.6, 0.5, 0.0, 51.666666666666664, 45.0, 20.0, 35.0, 20.0, 20.0, 60.0, 1.4, 1.6, 0.0, 3.0, 'H']


In [11]:
columns = ['GAME_ID', 'GAME_DATE', 'SEASON', 'HOME_TEAM', 'AWAY_TEAM', 'HOME_ODDS', 'AWAY_ODDS', 'DRAW_ODDS',
           'HOME_PTS_PCT', 'HOME_WIN_PCT', 'HOME_DRAW_PCT', 'HOME_LOSS_PCT', 'HOME_HOME_WIN_PCT', 'HOME_HOME_DRAW_PCT', 'HOME_HOME_LOSS_PCT', f'HOME_SCORED_LAST_{n_last_games}', f'HOME_CONCEDED_LAST_{n_last_games}', f'HOME_HOME_SCORED_LAST_{n_last_games}', f'HOME_HOME_CONCEDED_LAST_{n_last_games}',
           'AWAY_PTS_PCT', 'AWAY_WIN_PCT', 'AWAY_DRAW_PCT', 'AWAY_LOSS_PCT', 'AWAY_AWAY_WIN_PCT', 'AWAY_AWAY_DRAW_PCT', 'AWAY_AWAY_LOSS_PCT', f'AWAY_SCORED_LAST_{n_last_games}', f'AWAY_CONCEDED_LAST_{n_last_games}', f'AWAY_AWAY_SCORED_LAST_{n_last_games}', f'AWAY_AWAY_CONCEDED_LAST_{n_last_games}',
           'OUTCOME']
data_df = pd.DataFrame(data_model, columns=columns)

In [12]:
data_df.tail()

,GAME_ID,GAME_DATE,SEASON,HOME_TEAM,AWAY_TEAM,HOME_ODDS,AWAY_ODDS,DRAW_ODDS,HOME_PTS_PCT,HOME_WIN_PCT,...,AWAY_DRAW_PCT,AWAY_LOSS_PCT,AWAY_AWAY_WIN_PCT,AWAY_AWAY_DRAW_PCT,AWAY_AWAY_LOSS_PCT,AWAY_SCORED_LAST_5,AWAY_CONCEDED_LAST_5,AWAY_AWAY_SCORED_LAST_5,AWAY_AWAY_CONCEDED_LAST_5,OUTCOME
2291,838156,2022-07-17 11:00:00,2022,Juventude,Goias,2.04,4.10,3.14,25.000000,12.50,...,31.25,37.50,25.000000,12.500000,62.500000,0.8,1.0,0.000000,1.000000,D
2292,838153,2022-07-17 16:00:00,2022,Sao Paulo,Fluminense,2.53,3.11,3.03,47.916667,31.25,...,18.75,31.25,42.857143,28.571429,28.571429,1.8,0.2,0.500000,0.000000,D
2293,838155,2022-07-17 18:00:00,2022,Botafogo RJ,Atletico-MG,4.94,1.78,3.49,43.750000,37.50,...,43.75,12.50,28.571429,57.142857,14.285714,1.4,0.6,1.000000,0.500000,A
2294,838158,2022-07-17 18:00:00,2022,Atletico GO,Fortaleza,2.07,3.80,3.27,35.416667,25.00,...,31.25,56.25,14.285714,0.000000,85.714286,1.2,1.6,1.666667,2.666667,A
2295,838151,2022-07-17 19:00:00,2022,America MG,Bragantino,2.77,2.70,3.11,37.500000,31.25,...,37.50,31.25,14.285714,57.142857,28.571429,2.4,1.8,2.000000,3.000000,A


In [13]:
def parse_df_to_csv(dataframe, path, filename):
    if not os.path.exists(path):
        os.makedirs(path)
    dataframe.to_csv("{}/{}".format(path, filename))

In [14]:
parse_df_to_csv(data_df, 'data', '{}-{}.csv'.format(start_season, end_season))